## $\textbf{Importing Necessary Libraries}$

In [56]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns
import warnings

#%matplotlib qt
matplotlib.rcParams['backend'] = 'Qt5Agg'
warnings.filterwarnings('ignore')

## $\textbf{Defining All Functions Below}$

In [137]:
#Theoretical Formulas are first defined -----------------------------

def s(omega_m):
    return ((1-omega_m)/omega_m)**(1/3)

def eta(a,omega_m):
    term_1 = (1/a**(4)) - (0.1540*s(omega_m)/(a**(3))) + (0.4304*(s(omega_m)**2)/(a**(2)))
    term_2 = (0.19097*(s(omega_m)**3)/(a**(1))) + (0.066941*(s(omega_m)**4))
    
    return 2*np.sqrt((s(omega_m)**3) + 1)*((term_1 + term_2)**(-1/8))

## Luminosity Distance -----------------------------

def lum_dist(c,h,z,omega_m):
    term_1 = eta(1,omega_m)
    term_2 = eta((1/(1+z)),omega_m)
    H0 = 100*h
    
    return (c*(1+z)/H0)*(term_1 - term_2)

## Distance Modulus -----------------------------

def dist_mod(c,h,z,omega_m):
    mu = 25 - (5*np.log10(h)) + (5*np.log10(lum_dist(c,1,z,omega_m)))
    return mu

# Likelihood is now defined! This is actually the "Log Likelihood" ------------------
def likelihood(c,h,z,omega_m,mu_obs,cov_inv):
    expo_val = 0
    
    if h <= 0.0 or omega_m <= 0.0 or h >= 1.0 or omega_m >= 1.0:
        return -np.inf
    else:
        mu_theo = dist_mod(c,h,z,omega_m)
    
        for i in range(len(z_vals)):
            for j in range(len(z_vals)):
                expo_val += (mu_obs[i]-mu_theo[i])*cov_inv[i,j]*(mu_obs[j]-mu_theo[j])
    
        return -0.5*expo_val


# Markov Chain Monte Carlo - Metropolis Hastings Algorithm is defined below ---------

def mcmh(init_h,init_o,sigma,n_steps,c,z_vals,mu_vals_obs,covar_inv,likelihood):
    burn_in = int(0.1*n_steps)
    
    h_vals = np.zeros(n_steps)
    omega_vals = np.zeros(n_steps)

    h_vals[0] = init_h
    omega_vals[0] = init_o
    n_accept = 0

    for i in range(1,n_steps):
        current_h = h_vals[i-1]
        proposed_h = np.random.normal(current_h,sigma)

        current_omega = omega_vals[i-1]
        proposed_omega = np.random.normal(current_omega,sigma)

        num = likelihood(c,proposed_h,z_vals,proposed_omega,mu_vals_obs,covar_inv)
        den = likelihood(c,current_h,z_vals,current_omega,mu_vals_obs,covar_inv)

        prob_accept = min(1,np.exp(-den+num))
        alpha = np.random.rand(1)[0]

        if alpha <= prob_accept: #Accept the probability
            h_vals[i] = proposed_h
            omega_vals[i] = proposed_omega
            n_accept += 1
        elif alpha > prob_accept:
            h_vals[i] = current_h
            omega_vals[i] = current_omega
    
    final_omega_vals = omega_vals#[burn_in:]
    final_h_vals = h_vals#[burn_in:]
    
    return final_omega_vals,final_h_vals, n_accept/n_steps

## $\textit{Part 1: Plotting observed data}$

In [3]:
c = 3*(10**5)

omega_m_1 = 1.0
omega_m_2 = 0.5
omega_m_3 = 0.3

h1 = 1.0
h2 = 0.4
h3 = 0.7

In [208]:
data = np.loadtxt("file1.txt")

z_vals = data[:,0]
z_vals_obs = data[:,0]
mu_vals_obs = data[:,1]


mu_vals_obs_1 = dist_mod(c,h1,z_vals,omega_m_1)
mu_vals_obs_2 = dist_mod(c,h2,z_vals,omega_m_2)
mu_vals_obs_3 = dist_mod(c,h3,z_vals,omega_m_3)

plt.figure(figsize=(14,8),dpi=120)

plt.plot(z_vals_obs,mu_vals_obs,'bo',ms=5,alpha=0.5)

plt.plot(z_vals_obs,mu_vals_obs_1,ls = '--',color='orange',
         label="h = %.2f and $\Omega_m$ = %.2f" %(h1,omega_m_1))

plt.plot(z_vals_obs,mu_vals_obs_2,ls = '--',color='green',
         label="h = %.2f and $\Omega_m$ = %.2f" %(h2,omega_m_2))

plt.plot(z_vals_obs,mu_vals_obs_3,color='red',
         label="h = %.2f and $\Omega_m$ = %.2f" %(h3,omega_m_3))

plt.xlabel("Redshift $- z$",size=13,alpha=0.85)
plt.ylabel("Distance Modulus $- \mu$",size=13,alpha=0.85)
plt.title("Observed Distance Modulus of Supernova Ia",size=18,alpha=0.9)

plt.xticks(size=12,alpha=0.7)
plt.yticks(size=12,alpha=0.7)

plt.minorticks_on()

plt.legend(facecolor='lightcyan',loc='lower right',fontsize=15)
plt.grid(alpha=0.4,color='grey', linestyle='--', linewidth=0.5)

plt.savefig("dist_mod.pdf",format='pdf')

## $\textit{Part 2: Perfoming Paramter Estimation and evaluating values} $

In [30]:
data = np.loadtxt("file1.txt")
covar = np.loadtxt("file_cov.txt")

z_vals = data[:,0]
mu_vals_obs = data[:,1]
covar = covar.reshape(31,31)
covar_inv = np.linalg.inv(covar)

n_steps = 1000
c = 3*(10**5)
sigma = 0.01

omega_vals_1,h_vals_1,ar_1 = mcmh(0.95,0.05,sigma,n_steps,
                                  c,z_vals,mu_vals_obs,covar_inv,likelihood)
omega_vals_2,h_vals_2,ar_2 = mcmh(0.05,0.05,sigma,n_steps,
                                  c,z_vals,mu_vals_obs,covar_inv,likelihood)
omega_vals_3,h_vals_3,ar_3 = mcmh(0.05,0.95,sigma,n_steps,
                                  c,z_vals,mu_vals_obs,covar_inv,likelihood)
omega_vals_4,h_vals_4,ar_4 = mcmh(0.95,0.95,sigma,n_steps,
                                  c,z_vals,mu_vals_obs,covar_inv,likelihood)

/tmp/ipykernel_1865832/2253678544.py:65: RuntimeWarning: overflow encountered in exp
  prob_accept = min(1,np.exp(-den+num))


### $\text{Plotting and animating the data}$

In [61]:
x1_data = omega_vals_1
y1_data = h_vals_1

x2_data = omega_vals_2
y2_data = h_vals_2

x3_data = omega_vals_3
y3_data = h_vals_3

x4_data = omega_vals_4
y4_data = h_vals_4

fig, ax = plt.subplots(figsize=(14,8),dpi=120)

ax.plot(x1_data[0], y1_data[0], 'bo',alpha=.5, ms=4,
        label="RW 1 - ($h_0=0.95,\Omega_{m0}=0.05$) ")

ax.plot(x2_data[0], y2_data[0], 'ro',alpha=.5, ms=4,
        label="RW 2 - ($h_0=0.05,\Omega_{m0}=0.05$) ")

ax.plot(x3_data[0], y3_data[0], 'go',alpha=.5, ms=4,
        label="RW 3 - ($h_0=0.05,\Omega_{m0}=0.95$) ")

ax.plot(x4_data[0], y4_data[0],'o',color='orange',alpha=.5, ms=4,
        label="RW 4 - ($h_0=0.95,\Omega_{m0}=0.95$) ")

ax.set_xlabel("$h$ Values $\longrightarrow$",size=13,alpha=0.85)
ax.set_ylabel("$\Omega_m$ Values $\longrightarrow$",size=13,alpha=0.85)

ax.set_title("Parameter Estimation of $h$ and $\Omega_m$",size=18,alpha=0.9)

ax.set_xlim(0,1) 
ax.set_ylim(0,1) 

plt.minorticks_on()
plt.legend(fontsize=10,facecolor='lightcyan')
plt.pause(2)

for i in range(1, len(x1_data)):
    ax.plot(x1_data[i], y1_data[i], 'bo',alpha=.5, ms=3,
            label="RW 1 - ($h_0=0.95,\Omega_{m0}=0.05$) ")

    ax.plot(x2_data[i], y2_data[i], 'ro',alpha=.5, ms=3,
            label="RW 2 - ($h_0=0.05,\Omega_{m0}=0.05$) ")

    ax.plot(x3_data[i], y3_data[i], 'go',alpha=.5, ms=3,
            label="RW 3 - ($h_0=0.05,\Omega_{m0}=0.95$) ")

    ax.plot(x4_data[i], y4_data[i],'o',color='orange',alpha=.5, ms=3,
            label="RW 4 - ($h_0=0.95,\Omega_{m0}=0.95$) ")
    
    ax.plot([x1_data[i-1], x1_data[i]], [y1_data[i-1], y1_data[i]],
            'b-',lw=1.5,alpha=.5)
    
    ax.plot([x2_data[i-1], x2_data[i]], [y2_data[i-1], y2_data[i]],
            'r-',lw=1.5,alpha=.5)
    
    ax.plot([x3_data[i-1], x3_data[i]], [y3_data[i-1], y3_data[i]],
            'g-',lw=1.5,alpha=.5)
    
    ax.plot([x4_data[i-1], x4_data[i]], [y4_data[i-1], y4_data[i]],
            '-',color='orange',lw=1.5,alpha=.5)
    
    if i == int(0.5*len(x1_data)):
        plt.axvline(x=0.3,color='k',ls='--',lw=1.5,alpha=0.5)
        plt.axhline(y=0.7,color='k',ls='--',lw=1.5,alpha=0.5)
        
    #plt.pause(0.00001) #Uncomment this line to see animated plots

sns.kdeplot(x1_data, y1_data, fill=True, 
            common_norm=False, palette="crest",alpha=0.35)

sns.kdeplot(x2_data, y2_data, fill=True, 
            common_norm=False, palette="crest",alpha=0.35)

sns.kdeplot(x3_data, y3_data, fill=True, 
            common_norm=False, palette="crest",alpha=0.35)

sns.kdeplot(x4_data, y4_data, fill=True, 
            common_norm=False, palette="crest",alpha=0.35)


<AxesSubplot:title={'center':'Parameter Estimation of $h$ and $\\Omega_m$'}, xlabel='$h$ Values $\\longrightarrow$', ylabel='$\\Omega_m$ Values $\\longrightarrow$'>

### $\text{Plotting Histograms of } h \text{ and } \Omega_m$

In [282]:
def gaussian(x,mean,sigma):
    value = (1/np.sqrt(2*np.pi*(sigma**2)))*np.exp(-((x-mean)**2)/(2*(sigma**2)))
    return value

burn_in = int(0.30*n_steps)

# H values

h_vals_1_burn = h_vals_1[burn_in:]
h_vals_2_burn = h_vals_2[burn_in:]
h_vals_3_burn = h_vals_3[burn_in:]
h_vals_4_burn = h_vals_4[burn_in:]

mean_1_h = np.mean(h_vals_1_burn)
mean_2_h = np.mean(h_vals_2_burn)
mean_3_h = np.mean(h_vals_3_burn)
mean_4_h = np.mean(h_vals_4_burn)

net_mean_h = np.mean([mean_1_h,mean_2_h,mean_3_h,mean_4_h])

std_1_h = np.std(h_vals_1_burn)
std_2_h = np.std(h_vals_2_burn)
std_3_h = np.std(h_vals_3_burn)
std_4_h = np.std(h_vals_4_burn)

#Histogram of h

plt.figure(figsize=(14,8),dpi=120)

plt.hist(h_vals_1_burn,bins=50,color='blue',alpha=0.7,label='Random Walker 1')
plt.hist(h_vals_2_burn,bins=50,color='red',alpha=0.6,label='Random Walker 2')
plt.hist(h_vals_3_burn,bins=50,color='green',alpha=0.5,label='Random Walker 3')
plt.hist(h_vals_4_burn,bins=50,color='orange',alpha=0.4,label='Random Walker 4')

plt.axvline(x=net_mean_h,color='k',ls='--',alpha=0.5)

plt.xlabel("$h$ Values $\longrightarrow$",size=13,alpha=0.85)
plt.ylabel("Frequency $\longrightarrow$",size=13,alpha=0.85)
plt.title("Distribution of $h$ values",size=18,alpha=0.9)

plt.xticks(size=13,alpha=0.5)
plt.yticks(size=13,alpha=0.5)
plt.minorticks_on()

plt.legend(fontsize=12,facecolor='lightcyan')
#plt.savefig("histo_h.pdf",format='pdf')

#OMEGA_M

omega_vals_1_burn = omega_vals_1[burn_in:]
omega_vals_2_burn = omega_vals_2[burn_in:]
omega_vals_3_burn = omega_vals_3[burn_in:]
omega_vals_4_burn = omega_vals_4[burn_in:]

mean_1_omega = np.mean(omega_vals_1_burn)
mean_2_omega = np.mean(omega_vals_2_burn)
mean_3_omega = np.mean(omega_vals_3_burn)
mean_4_omega = np.mean(omega_vals_4_burn)

net_mean_omega = np.mean([mean_1_omega,mean_2_omega,mean_3_omega,mean_4_omega])

std_1_omega = np.std(omega_vals_1_burn)
std_2_omega = np.std(omega_vals_2_burn)
std_3_omega = np.std(omega_vals_3_burn)
std_4_omega = np.std(omega_vals_4_burn)

#Histogram of Omega
plt.figure(figsize=(14,8),dpi=120)

plt.hist(omega_vals_1_burn,bins=50,color='blue',alpha=0.7,label='Random Walker 1')
plt.hist(omega_vals_2_burn,bins=50,color='red',alpha=0.6,label='Random Walker 2')
plt.hist(omega_vals_3_burn,bins=50,color='green',alpha=0.5,label='Random Walker 3')
plt.hist(omega_vals_4_burn,bins=50,color='orange',alpha=0.4,label='Random Walker 4')

plt.axvline(x=net_mean_omega,color='k',ls='--',alpha=0.5)

plt.xlabel("$\Omega_m$ Values $\longrightarrow$",size=13,alpha=0.85)
plt.ylabel("Frequency $\longrightarrow$",size=13,alpha=0.85)
plt.title("Distribution of $\Omega_m$ values",size=18,alpha=0.9)

plt.xticks(size=13,alpha=0.5)
plt.yticks(size=13,alpha=0.5)
plt.minorticks_on()

plt.legend(fontsize=12,facecolor='lightcyan')
#plt.savefig("histo_omega.pdf",format='pdf')

### $\text{Evaluating statistics}$

In [246]:
final_h_vals = h_vals_2_burn
final_omega_vals = omega_vals_2_burn

print("The acceptance ratio for Random Walker 2 is: %.3f" %ar_2)
print("\n")
# Evaluating Mean
final_h_mean = np.mean(final_h_vals)
final_omega_mean = np.mean(final_omega_vals)

# Evaluating Standard Deviation and Variance
final_h_std = np.std(final_h_vals)
final_omega_std = np.std(final_omega_vals)

final_h_var = final_h_std**2
final_omega_var = final_omega_std**2

#Evaluating covariance
covariance = np.cov(final_h_vals,final_omega_vals)

print("The mean h-value is: %.3f" %final_h_mean)
print("The mean omega-value is: %.3f" %final_omega_mean)
print("\n")
print("The variance of h is: %.6f" %final_h_var)
print("The variance of omega is: %.6f" %final_omega_var)
print("\n")
print("The Covariance matrix is given by:\n ", covariance)

The acceptance ratio for Random Walker 2 is: 0.345


The mean h-value is: 0.705
The mean omega-value is: 0.281


The variance of h is: 0.000052
The variance of omega is: 0.000554


The Covariance matrix is given by:
  [[ 5.23691621e-05 -6.47581889e-05]
 [-6.47581889e-05  5.55136563e-04]]


In [253]:
mu_vals_obs_final = dist_mod(c,final_h_mean,z_vals,final_omega_mean)

fig,ax = plt.subplots(figsize=(14,8),dpi=120)
ax.set_facecolor('whitesmoke')

ax.plot(z_vals_obs,mu_vals_obs,'ko',ms=5,alpha=0.8)

ax.plot(z_vals_obs,mu_vals_obs_final,ls = '--',color='red',
         label="h = %.3f and $\Omega_m$ = %.3f" %(final_h_mean,final_omega_mean))

ax.set_xlabel("Redshift $- z$",size=13,alpha=0.85)
ax.set_ylabel("Distance Modulus $- \mu$",size=13,alpha=0.85)
ax.set_title("Observed Distance Modulus of Supernova Ia",size=18,alpha=0.9)

plt.xticks(size=12,alpha=0.7)
plt.yticks(size=12,alpha=0.7)

plt.minorticks_on()

plt.legend(facecolor='lightcyan',loc='lower right',fontsize=15)
plt.grid(alpha=0.4,color='grey', linestyle='--', linewidth=0.5)

plt.savefig("final_plot.pdf",format='pdf')

### $\text{Variation of Acceptance Probability with size of proposal distribution} $ $\sigma$

In [254]:
c = 3*(10**5)
n_steps = 10000
burn_in = int(0.15*n_steps)

sigma_2 = np.linspace(0.0000001,0.0000009,9,endpoint=True)
sigma_1 = np.linspace(0.000001,0.000009,9,endpoint=True)
sigma_ = np.linspace(0.00001,0.00009,9,endpoint=True)
sigma0 = np.linspace(0.0001,0.0009,9,endpoint=True)
sigma1 = np.linspace(0.001,0.009,9,endpoint=True)
sigma2 = np.linspace(0.01,0.09,9,endpoint=True)
sigma3 = np.linspace(0.1,0.9,9,endpoint=True)
sigma4 = np.linspace(1,10,10,endpoint=True)
sigma5 = np.linspace(10,100,10,endpoint=True)

sigma = np.concatenate((sigma_2,sigma_1,sigma_,sigma0,sigma1,sigma2,sigma3,sigma4,sigma5))
h_init = 0.15
omega_init = 0.75

acceptance_ratio = np.zeros(len(sigma))
mean_h_array = np.zeros(len(sigma))
mean_omega_array = np.zeros(len(sigma))

for i in range(len(sigma)):
    
    d1,d2,accept_temp = mcmh(h_init,omega_init,sigma[i],n_steps,
                                  c,z_vals,mu_vals_obs,covar_inv,likelihood)
    
    acceptance_ratio[i] = accept_temp
    mean_omega_array[i] = np.mean(d1[burn_in:])
    mean_h_array[i] = np.mean(d2[burn_in:])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82


In [280]:
fig, ax = plt.subplots(figsize=(14,8),dpi=120)

ax.set_facecolor("whitesmoke")
ax.plot(sigma,acceptance_ratio,'bo-',alpha=0.8,ms=7,lw=1.5,markerfacecolor='red')
#plt.axvline(x=sigma[4],color='r',alpha=0.5,linestyle='--')
#plt.axhline(y=acceptance_ratio[4],color='r',alpha=0.5,linestyle='--')

ax.set_xscale("log")
plt.minorticks_on()
plt.xticks(size=13)
plt.yticks(size=13)

ax.set_xlabel("$\sigma \longrightarrow$",size=15)
ax.set_ylabel("Acceptance Probability $A$ $\longrightarrow$",size=15)

ax.set_title("Variation of Acceptance Probability $A$ with $\sigma$",size=18)
plt.grid(alpha=0.4,color='grey', linestyle='--', linewidth=0.5)

plt.savefig("accpt_prob_sigma.pdf",format='pdf')

In [281]:
info = "Region of interest"
fig, ax = plt.subplots(figsize=(14,8),dpi=120)
ax.set_facecolor("whitesmoke")

ax.plot(sigma,mean_h_array,'bo-',alpha=0.8,lw=1.8,label='$h$',markerfacecolor='red')
ax.plot(sigma,mean_omega_array,'go-',alpha=0.8,lw=1.8,label='$\Omega_m$',markerfacecolor='black')
plt.axvline(x=1.e-3,color='k',alpha=0.5,linestyle='--')
plt.axvline(x=1.e0,color='k',alpha=0.5,linestyle='--')
plt.text(5.e-3, 0.8, info, fontsize=12, bbox=dict(facecolor='salmon', alpha=0.5,pad=8))

ax.set_xscale("log")
plt.minorticks_on()
plt.xticks(size=13)
plt.yticks(size=13)

ax.set_xlabel("$\sigma \longrightarrow$",size=15)
ax.set_ylabel("Mean $h $ and $ \Omega_m$ $\longrightarrow$",size=15)
ax.set_title("Variation of $h $ and $ \Omega_m$ with $\sigma$",size=18)

plt.grid(alpha=0.4,color='grey', linestyle='--', linewidth=0.5)
plt.legend(fontsize=12,loc='upper right')

plt.savefig("mean_omega_sigma.pdf")

### $\text{Small Distribution and Large Distribution}$

In [226]:
n_steps = 1000
c = 3*(10**5)
sigma_small = 1.e-4
sigma_large = 100

h_init = 0.05
omega_init = 0.95

omega_vals_small,h_vals_small,ar_small = mcmh(h_init,omega_init,sigma_small,n_steps,
                                  c,z_vals,mu_vals_obs,covar_inv,likelihood)
omega_vals_large,h_vals_large,ar_large = mcmh(h_init,omega_init,sigma_large,n_steps,
                                  c,z_vals,mu_vals_obs,covar_inv,likelihood)

In [239]:
fig, ax = plt.subplots(figsize=(14,8),dpi=120)

ax.set_facecolor('whitesmoke')
ax.scatter(omega_vals_small,h_vals_small,s=5,alpha=0.5,color='blue')
ax.set_xlabel("$\Omega_m$ Values $\longrightarrow$",size=15)
ax.set_ylabel("$h$ Values $\longrightarrow$",size=15)
ax.set_title("Random Walker for small distribution - $\sigma = 10^{-4}$",size=15)
plt.minorticks_on()
plt.xticks(size=13)
plt.yticks(size=13)
plt.grid(alpha=0.4,color='k', linestyle='--', linewidth=0.5)

plt.savefig("small_dist.pdf")

fig, ax = plt.subplots(figsize=(14,8),dpi=120)

ax.set_facecolor('whitesmoke')
ax.scatter(omega_vals_large,h_vals_large,s=20,alpha=0.5,color='red')
ax.set_xlabel("$\Omega_m$ Values $\longrightarrow$",size=15)
ax.set_ylabel("$h$ Values $\longrightarrow$",size=15)
ax.set_title("Random Walker for small distribution - $\sigma = 100$",size=15)
plt.minorticks_on()
plt.xticks(size=13)
plt.yticks(size=13)
plt.grid(alpha=0.4,color='k', linestyle='--', linewidth=0.5)

plt.savefig("large_dist.pdf")